In [1]:
import pandas as pd
import tensorflow as tf
import rdkit
from main import create_input, create_model, predict
from argparse import ArgumentParser
from molgraph import single_mol_from_supergraph
import numpy as np

Using backend: tensorflow


# Run predictive model

In [2]:
parser = ArgumentParser()
parser.add_argument('-predict', action="store_true", default=True)
parser.add_argument('-watsoneq', action='store_true', default=False)
parser.add_argument('-K_fold', action='store_true', default=False)
parser.add_argument('-maxatoms', type=int, default=64)
parser.add_argument('-lr', type=float, default=5.0e-4)
parser.add_argument('-epoch', type=int, default=100)
parser.add_argument('-batchsize', type=int, default=256)
parser.add_argument('-layers', type=int, default=5)
parser.add_argument('-heads', type=int, default=5)
parser.add_argument('-residcon', action="store_true", default=True)
parser.add_argument('-explicitH', action="store_true", default=False)
parser.add_argument('-dropout', type=float, default=0.0)
parser.add_argument('-modelname', type=str, default='best_211007')
parser.add_argument('-num_hidden', type=int, default=32)
parser.add_argument('-train_only', action="store_true", default=False)
parser.add_argument('-sw_thr', type=float, default=0.0)
parser.add_argument('-sw_decay', type=int, default=1)
parser.add_argument('-loss', type=str, default='kl_div_normal')
args = parser.parse_args('')

In [3]:
df = pd.read_csv('data/Data_211005.csv')
tmp = []
#for atom feature t-SNE and atom attention weights (no temperature dependence)
for smi in df.smiles.unique():
    tmp.append([smi,298.15])
pd.DataFrame(tmp).to_csv('molecules_to_predict.csv', header=['smiles','temperature'], index=False)


#for temperature dependence
#pd.DataFrame(df[['smiles','Train/Valid/Test','temperature','HoV (kJ/mol)']]).to_csv('molecules_to_predict.csv', index=False)


In [4]:
device ='/cpu:0'
data = pd.read_csv('molecules_to_predict.csv')
data['total_atoms'] = [ rdkit.Chem.MolFromSmiles(smi).GetNumHeavyAtoms() for smi in data.smiles]
INPUT = create_input(data, device, args)
data, num_mols, T, Graphs, seg = INPUT
atom_feat_dim = Graphs.ndata['feat'].shape[-1]
model, model_name = create_model(args, atom_feat_dim, '')
################
Predicted_HoV, atom_features_each_layer, Attention_each_layer, T_part, updated_T = predict(model, Graphs.ndata['feat'], Graphs, \
                                                    seg, args.maxatoms, T, '', num_mols, mu_s_NLR=[])
pred_mean, pred_stddev = Predicted_HoV[:, 0], Predicted_HoV[:, 1]
data['Predicted'] = pred_mean.numpy()
data['ML_unc'] = pred_stddev.numpy()

0
1
2
3
4


In [6]:
#If full DB with temperature dependence is tested
#valid = data[data['Train/Valid/Test'] == 'Valid']
#np.abs(valid.Predicted - valid['HoV (kJ/mol)']).mean()
#train = data[data['Train/Valid/Test'] == 'Train']
#np.abs(train.Predicted - train['HoV (kJ/mol)']).mean()
#test = data[data['Train/Valid/Test'] == 'Test']
#np.abs(test.Predicted - test['HoV (kJ/mol)']).mean()

# Run t-SNE

In [7]:
indices_nonzeros = []

start = 0
for tot_atom in data.total_atoms:
    indices_nonzeros += list(range(start, start+tot_atom))
    start += 64

#initial_onehot = Graphs.ndata['feat'].numpy()[indices_nonzeros]
atom_features_layer1 = atom_features_each_layer[0].numpy()[indices_nonzeros]
atom_features_layer3 = atom_features_each_layer[2].numpy()[indices_nonzeros]
atom_features_layer5 = atom_features_each_layer[4].numpy()[indices_nonzeros]

len(atom_features_layer1)

94200

In [8]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
pipe = Pipeline(steps=(
    ('PCA', PCA(n_components=10)),
    ('TSNE', TSNE(n_components=2))
))

#xt_onehot = pipe.fit_transform(initial_onehot)
xt_1 = pipe.fit_transform(atom_features_layer1)
xt_3 = pipe.fit_transform(atom_features_layer3)
xt_5 = pipe.fit_transform(atom_features_layer5)

In [9]:
feat_vectors_2d = np.concatenate([xt_1,xt_3,xt_5], axis=1)
df = pd.DataFrame(feat_vectors_2d, columns = ['xt_1_x', 'xt_1_y',
                                         'xt_3_x', 'xt_3_y',
                                        'xt_5_x', 'xt_5_y'])

# Attention score analysis

In [10]:
Att_each_layer_results = []

for l in range(len(Attention_each_layer)):
    A_per_each_mol, src_per_each_mol, dst_per_each_mol = single_mol_from_supergraph(Graphs, 
                                                                                    atom_features_each_layer[l], 
                                                                                    Attention_each_layer[l], 
                                                                                    Max_atoms = 64)
    Att_each_layer_results.append([A_per_each_mol, src_per_each_mol, dst_per_each_mol])

In [11]:
def make_attention_matrix(A, src, dst):
    A_matrix = np.zeros((max(src) + 1, max(src) +1 ))
    for i in range(len(A)):
        A_matrix[src[i], dst[i]] = A[i]
    return A_matrix

In [12]:
unique_smiles = list(data.smiles.unique())
unique_smiles_numatoms = [rdkit.Chem.MolFromSmiles(smi).GetNumHeavyAtoms() for smi in unique_smiles]

atom_indices_each_molecule = []
each_atom_belongs_to_which_smiles = []

for i in range(len(unique_smiles)):
    atom_indices_each_molecule += list(range(unique_smiles_numatoms[i]))
    each_atom_belongs_to_which_smiles += [unique_smiles[i]] * unique_smiles_numatoms[i]

print(len(atom_indices_each_molecule), len(each_atom_belongs_to_which_smiles))
    
Aij_rowsum_all_layers = []
for i in range(len(Att_each_layer_results)):
    Aij_rowsum = []
    
    Att_ith = Att_each_layer_results[i]
    A_per_each_mol, src_per_each_mol, dst_per_each_mol = Att_ith
    
    for molecule_index in range(len(A_per_each_mol)):
        A, src, dst = A_per_each_mol[molecule_index], \
                      src_per_each_mol[molecule_index], \
                      dst_per_each_mol[molecule_index]
        
        A_matrix = make_attention_matrix(A, src, dst)
        
        if unique_smiles_numatoms[molecule_index] != len(A_matrix):
            print('???????????????')
        
        Aij_rowsum_one_molecule = []
        for atom_index in range(len(A_matrix)):
            #Aij_rowsum.append(np.sum(A_matrix[atom_index]) / np.count_nonzero(A_matrix[atom_index]))
            Aij_rowsum_one_molecule.append(np.sum(A_matrix[atom_index]) / len(A_matrix))
        Aij_rowsum_one_molecule = [ x / max(Aij_rowsum_one_molecule) for x in Aij_rowsum_one_molecule]
        
        Aij_rowsum += Aij_rowsum_one_molecule
    Aij_rowsum_all_layers.append(Aij_rowsum)

94200 94200


In [13]:
df['smiles'] = each_atom_belongs_to_which_smiles
df['atom_index'] = atom_indices_each_molecule

for i in range(len(Aij_rowsum_all_layers)):
    df['Att_'+str(i+1)] = Aij_rowsum_all_layers[i]

# Atom type/substructure analysis

In [14]:
from rdkit import Chem

def substruc_search(smiles, atom_index, radius=1):
    m = Chem.MolFromSmiles(smiles)
    
    env = Chem.FindAtomEnvironmentOfRadiusN(m,radius,int(atom_index))
    amap={}
    submol=Chem.PathToSubmol(m,env,atomMap=amap)
    
    
    mol_smarts=Chem.MolToSmarts(submol)
    
    return mol_smarts

In [15]:
smarts_radius_1_each_atom = []
for _, row in df.iterrows():
    a_ind, smi = row['atom_index'], row['smiles']
    smarts = substruc_search(smi, a_ind)
    smarts_radius_1_each_atom.append(smarts)
    
print(len(smarts_radius_1_each_atom))

94200


In [16]:
df['smarts'] = smarts_radius_1_each_atom

In [17]:
def atom_type(smi, a_ind):
    mol = rdkit.Chem.MolFromSmiles(smi)
        
    atom = mol.GetAtomWithIdx(a_ind)    
    
    atom_type = ''
    atom_type += atom.GetSymbol()

    num_bonds = atom.GetDegree() + atom.GetTotalNumHs()

    if atom.GetSymbol() == 'C':
        if num_bonds == 4:
            atom_type += 'sp3'
        elif num_bonds == 3:
            atom_type += 'sp2'
        elif num_bonds <= 2:
            atom_type += 'sp'
    elif atom.GetSymbol() == 'O':
        if num_bonds == 2:
            atom_type += 'sp3'
        elif num_bonds == 1:
            atom_type += 'sp2'
        else:
            print("?!?!?!")

    if atom.GetIsAromatic():
        atom_type += '(aro)'
    elif atom.IsInRing():
        atom_type += '(ring)'

    if atom_type == 'Csp(aro)':
        atom_type = 'C(ring)'
    elif atom_type == 'Csp2(aro)':
        atom_type = 'C(aro)'
    elif atom_type == 'Osp3(aro)':
        atom_type = 'O(aro)'

    if atom_type == 'Csp(ring)' or atom_type == 'Csp2(ring)' or atom_type == 'Csp3(ring)':
        atom_type = 'C(ring)'

    if atom_type == 'Osp3':
        atom_type = '-O-'
    if atom_type == 'Osp2':
        atom_type = 'O='
    if atom_type == 'Osp3(ring)':
        atom_type = 'O(ring)'
        
    return atom_type

In [18]:
df['atom_type'] = [atom_type( row['smiles'], row['atom_index']) for _, row in df.iterrows() ]

In [19]:
df.to_csv('feat_vectors_2d+Att.csv', index=False)